In [1]:
# what kind of files do we have?
from glob import glob
from pathlib import Path
from collections import Counter

GLOB = '/kaggle/input/chessman-image-dataset/Chessman-image-dataset/Chess/*/*'
suffixes = Counter([Path(input_file).suffix for input_file in glob(pathname=GLOB)])

print(suffixes)

Counter({'.jpg': 465, '.png': 69, '.JPG': 10, '.jpeg': 7, '.fcgi': 2, '.php': 1, '.webp': 1, '.gif': 1})


We have some files that are not images and we need to be sure to ignore them.

In [2]:
from PIL import Image

this_file = '/kaggle/input/chessman-image-dataset/Chessman-image-dataset/Chess/Bishop/00000000.JPG'
data = Image.open(this_file).convert('RGB')

In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms

device = torch.device('cpu')
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1).to(device=device)
model.eval()

layer = model._modules.get('avgpool')

layer_output_size = 512

scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

item = normalize(to_tensor(scaler(data)))
items = list(item.unsqueeze(0))
images = torch.stack(items).to(device)
result = model(images)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 52.3MB/s]


In [4]:
result[0].detach().numpy()

array([-3.58456421e+00, -2.26107907e+00, -3.61414146e+00, -4.33785915e+00,
       -1.75912809e+00, -2.78984857e+00, -2.62123299e+00, -5.86968899e-01,
       -1.23100042e+00, -1.70607865e+00,  2.48540193e-02, -7.57809937e-01,
        4.59266305e-02, -6.40145540e-01,  4.04473960e-01, -1.23141909e+00,
       -5.36737680e-01, -2.66594470e-01,  1.00560212e+00,  1.45740223e+00,
       -1.70287287e+00, -1.22665620e+00, -2.44607520e+00, -1.37693775e+00,
       -3.64056754e+00, -9.86307204e-01, -2.17123938e+00, -2.66567111e+00,
        4.99795943e-01, -9.10017550e-01, -1.25512183e+00, -2.01697588e+00,
       -1.17628753e+00, -2.85059476e+00, -1.43012643e+00, -2.00303555e+00,
        1.18990445e+00, -8.18175852e-01, -1.33473361e+00, -1.97544277e-01,
       -1.20242643e+00,  5.03720462e-01, -1.18466258e+00, -1.15329790e+00,
       -1.58870757e+00,  4.07433718e-01, -3.64442974e-01, -3.63813472e+00,
       -2.03270268e+00, -2.31171036e+00, -1.79887283e+00, -6.32842124e-01,
       -7.33385444e-01,  